# MockUI Jupyter notebook to prototyping the LP-application using Plotly Dash

* Scope of prototyping  
  - Fetching input data from Excel
  - Fetching results data from GAMS .gdx-file 
  - Visualizing input and results data using Plotly Express
  - Setting up web application using Dash components

In [22]:
from typing import IO, Any
import sys
import os
import logging
import shutil
import locale
import time
import datetime as dt
from datetime import datetime, timedelta
import asyncio
from venv import create
import numpy as np
import pandas as pd
# import pyxlsb
import xlwings as xw
import GdxWrapper as gw
import gams.transfer as gtr
import plotly
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import Dash, html, dash_table, dcc, Output, Input, State
import dash_bootstrap_components as dbc
from dash.exceptions import PreventUpdate

pd.options.display.max_columns = None

appPath = r'C:\\GitHub\\23-4002-LPTool\\App'
if not appPath in sys.path:
    sys.path = [] + sys.path
print('\n'.join(sys.path))

from lpBase import LpBase, JobResultKind
LpBase.setAppRootPath(appPath)
logger = LpBase.initLogger('MecLpTool')

print(f'Timestamp: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')
for p in [plotly, dash, dcc, html, dbc, pd, np, xw]:
    print(f'{p.__name__:-<30}v{p.__version__}')

c:\GitHub\23-4002-LPTool\App\MockUI
C:\anaconda3
c:\anaconda3\envs\mbl\python311.zip
c:\anaconda3\envs\mbl\DLLs
c:\anaconda3\envs\mbl\Lib
c:\anaconda3\envs\mbl

C:\Users\MogensBechLaursen\AppData\Roaming\Python\Python311\site-packages
c:\anaconda3\envs\mbl\Lib\site-packages
c:\anaconda3\envs\mbl\Lib\site-packages\win32
c:\anaconda3\envs\mbl\Lib\site-packages\win32\lib
c:\anaconda3\envs\mbl\Lib\site-packages\Pythonwin
Timestamp: 2024-01-10 09:46:13
plotly------------------------v5.18.0
dash--------------------------v2.14.2
dash.dcc----------------------v2.12.1
dash.html---------------------v2.0.15
dash_bootstrap_components-----v1.5.0
pandas------------------------v2.1.4
numpy-------------------------v1.24.3
xlwings-----------------------v0.29.1


In [23]:
class StemData():

    _logger = LpBase.getLogger()

    def __init__(self, fileName: str = 'MecLPinput.xlsm'):
        """ 
        Initializes the StemData object. 
        Reads data from the excel file and stores it in a dictionary.
        A lazy implementation is not used due to the excessive load time of the Excel file.
        """

        # self.path = os.path.join('C:\\GitHub\\23-4002-LPTool\\Data\\MockUI', fileName)
        self.path = os.path.join('C:\\GitHub\\23-4002-LPTool\\Master', fileName)
        if not os.path.exists(self.path):
            print(f'Error: File {self.path} does not exist.')

        self.data = self.read_excel_data()

    def read_excel_data(self) -> dict[str, pd.DataFrame]:
        """
        Reads data from the excel file and returns a dictionary with the data.
        """
        # Read data from excel file
        data = dict()  # Key is table name, value is dataframe.
        xlapp = xw.App(visible=False, add_book=False)
        try:
            wb = xlapp.books.open(self.path, read_only=True)
            data['LpTables'] = wb.sheets['LPspec'].range('tblLpTables').options(pd.DataFrame, expand='table', index=False).value
            lpTables = data['LpTables']

            for i in range(len(lpTables)):
                tableName = lpTables.loc[i,'TableName']
                sheetName = lpTables.loc[i,'SheetName']
                rangeName = lpTables.loc[i,'RangeName']
                useIndex = lpTables.loc[i,'UseIndex']
                rowDim = int( lpTables.loc[i,'RowDim']) if useIndex else 0  
                colDim = int(lpTables.loc[i,'ColDim'])
                StemData._logger.info(f'Reading table {tableName} from sheet {sheetName} with range {rangeName}.')
                # logger.info(f'UseIndex={useIndex}, RowDim = {rowDim}, ColDim = {colDim}.')
                df = wb.sheets[sheetName].range(rangeName).options(pd.DataFrame, expand='table', index=rowDim, header=colDim).value
                data[tableName] = df
            wb.close()
            
        except Exception as e:
            StemData._logger.exception(f'Error reading excel file {self.path}.', exc_info=True)
        finally:
            xlapp.quit()

        return data

class ModelData():

    def __init__(self, fileName: str = 'MecLpMain.gdx'):
        """ Initializes the ModelData object. """
        global logger
        # self.path = os.path.join('C:\\GitHub\\23-4002-LPTool\\Data\\MockUI', fileName)
        self.path = os.path.join('C:\\GitHub\\23-4002-LPTool\\Master', fileName)
        if not os.path.exists(self.path):
            raise ValueError(f'File {self.path} does not exist.')

        self.Gsymbols = dict()  # Key is symbol name in lower case, value is GSymbolProxy instance.
        self.data = dict()      # Key is symbol name in lower case, value is dataframe of records.
        self.gw = gw.GdxWrapper(name='ModelData', pathFile=self.path, loggerName=logger.name)
        return

    def readSymbolAsDataFrame(self, symbolName: str, attrName: str = 'level') -> pd.DataFrame:
        """
        Reads data of a single GAMS symbol from the gdx file and returns a dataframe with the data.
        """
        # Read symbol data from gdx file
        gsym = gw.GSymbolProxy(symbolName, self.gw)
        symbolData = self.gw.getRecords(symbolName.lower(), attrName)
        if symbolData is None:
            return None
        
        self.Gsymbols[symbolName.lower()] = gsym
        self.data[symbolName.lower()] = symbolData

        return symbolData

    def __getitem__(self, symbolName: str) -> pd.DataFrame:
        """ Returns the dataframe with the given key. Lazy implementation."""
        # See: https://www.kdnuggets.com/2023/03/introduction-getitem-magic-method-python.html
        
        if symbolName.lower() not in self.Gsymbols:
            symbolData = self.readSymbolAsDataFrame(symbolName)
            if symbolData is None:
                logger.error(f'Symbol of name {symbolName} was not found.')
                return None
            self.data[symbolName] = symbolData

        return self.data[symbolName.lower()]
    
def createPivot(dfRecs: pd.DataFrame, indexName: str, columnNames: list[str], valueName: str,
                fillna: bool = True, createTimeColumn: bool = False, timeVector: list[float] = None) -> pd.DataFrame:
    """
    Creates a pivot table from a DataFrame of records e.g. of a GAMS symbol like parameter, variable, equation.
    Each column of dfRecs holds the values of a defining dimension of the symbol.
    One column holds the attribute of the symbol e.g. value, level, marginal, lower, upper.
    
    Parameters
    ----------
    dfRecs : pd.DataFrame
        Holds the records part of which will be used to compose the pivot table
    indexName : str
        Name of the column in dfRecs that should be index of the pivot table.
    columnNames : list[str]
        List of names of columns in dfRecs to constitute the columns of the pivot table.
        If columnNames has two or more members, the columns of the pivot table will be a 
        multiindex i.e. a tuple of each dimension's member value (name).
    valueName : str
        Name of the column in dfRecs whose values will fill the body of the pivot table.
    fillna : boolean, optional
        If True (default), NaN-values will be converted to zeros.
    createTimeColumn : boolean, optional
        If True (default) and index of pivot the column of name 'tt', the numeric part of 
        the index values will be converted to integers and stored in a new column named 'time',
        and the entire pivot table sorted ascendingly by this column.

    Raises
    ------
    ValueError
        Either one of indexName, columnNames or valueName was not found in dfRecs.

    Returns
    -------
    pivot : DataFrame
        The pivot table

    """
    if indexName is not None and not indexName in dfRecs.columns:
        raise ValueError(f'{indexName=} not found in columns of DataFrame dfRecs')
    
    for col in columnNames:
        if not col in dfRecs.columns:
            raise ValueError(f'Column {col=} not found in columns of DataFrame dfRecs')
    
    if not valueName in dfRecs.columns:
        raise ValueError(f'{valueName=} not found in columns of DataFrame dfRecs')
    
    pivot = dfRecs.pivot(index=indexName, columns=columnNames, values=valueName)
    
    if fillna:
        pivot = pivot.fillna(0.0)
        
    if createTimeColumn:
        # Assuming the index of pivot has members of kind 't'nnnn where n is a digit.
        if pivot.index.name != 'tt':
            raise ValueError(f'Pivot must have index of name "tt", but "{pivot.index.name}" was found')
            
        if timeVector is None:
            pivot['time'] = [int(tt[1:]) for tt in pivot.index]
        else:
            pivot['time'] = [timeVector[int(tt[1:]) - 1] for tt in pivot.index]

        pivot = pivot.sort_values(by=['time'])
    
    return pivot


### Classes and methods to execute the GAMS model

In [ ]:

# import class JobSpec from file JobSpec.py located in folder ../App/Engine
from jobSpec import JobSpec
from JobLib import CoreData 

parms = JobSpec.getDefaultMasterParms()
parms = JobSpec.getDefaultMasterParms()
jobSpec = JobSpec('name', 'desc', parms)



In [3]:

from Engine.JobSpec import JobSpec
from Engine.JobLib import CoreData 

def time2int(time: datetime) -> int:
    """ Converts a datetime object to an integer of kind 't'nnnn. """
    return int(time.strftime('%Y%m%d%H'+'00'))

def setup(jobSpec: JobSpec, rootDir, logger) -> None:
    """ Sets up the model prior to execution. """

    core = CoreData('None', rootDir, logger) 

    # Copy model files to the working directory.
    core.copyInputFiles(copyAllFiles=True)
    core.openExcelInputFile(visible=True)

    # Set master parameters of the model. Use master scenario 2 (default).
    core.setDefaultParms(2, 0, 0, 0, 0, 0)
    for key, value in jobSpec.masterParms.items():
        core.setParmMaster(key, value)

    return 

In [ ]:
rootDir = 'C:\\GitHub\\23-4002-LPTool'

allPlants = [   'MaNVak', 'MaVak', 'HoNVak', 'StVak', 'MaCool', 'MaCool2', 'MaAff1', 'MaAff2', 'MaBio', 'MaEk', 'MaNbk', 'MaNbKV', 'MaNEk', 'MaNhpAir', 'MaNhpPtX', 
                'HoNEk', 'HoNFlis', 'HoNhpAir', 'HoNhpArla', 'HoNhpBirn', 'HoNhpSew', 'HoGk', 'HoOk', 
                'StEk', 'StNEk', 'StNFlis', 'StNhpAir', 'StGk', 'StOk']

activePlants = ['MaNVak', 'MaVak', 'StVak', 'MaCool', 'MaCool2', 'MaAff1',
                'MaBio', 'MaEk', 'MaNbk', 'MaNhpAir',
                'HoNhpBirn', 'HoNhpSew', 'HoGk', 'HoOk', 
                'StEk', 'StGk', 'StOk']

jobSpec = JobSpec(
        timeStart    = datetime.fromisoformat('2024-01-09'),     # Start time of model planning horizon.
        duration     = 2 * 24,           # Duration [hour] of model planning horizon.
        hourBegin    = 577,              # Hour of day when historical data interval starts.
        resolBid     = 60,               # Resolution of bid day in minutes.
        resolDefault = 60,               # Resolution of other days in minutes.
        activePlants = activePlants      # List of active plants.
)

# setup(jobSpec, rootDir, logger)

core = CoreData('m01s01u00r00f00', rootDir, logger) 

# Copy model files to the working directory.
core.copyInputFiles(copyAllFiles=True)
core.openExcelInputFile(visible=True)

# Set master parameters of the model. Use master scenario 2 (default).
core.setDefaultParms(2, 0, 0, 0, 0, 0)
core.setParmMaster('OnUGlobalScen.HourBegin',             jobSpec.hourBegin)
# core.setParmMaster('OnUGlobalScen.HourEnd',               jobSpec.hourEnd)
core.setParmMaster('OnUGlobalScen.TimestampStart',        jobSpec.timeStart.strftime('%Y%m%d%H'+'00'))
core.setParmMaster('OnUGlobalScen.DurationPeriod',        jobSpec.DurationPeriod)
core.setParmMaster('OnUGlobalScen.TimeResolutionDefault', jobSpec.resolDefault)
core.setParmMaster('OnUGlobalScen.TimeResolutionBid',     jobSpec.resolBid)

for plant in jobSpec.activePlants:
        core.setParmMaster(f'OnUGlobalScen.{plant}', 1)


### Classes and methods to read GAMS model results from gdx

In [ ]:
logger = setupLogger('LpMockUI')

#region Reading data

readStemData = False
readModelData = True

if readStemData or readModelData:

    tbegin = time.perf_counter_ns()
    if readStemData:
        # Create StemData object
        logger.info('Reading StemData.')
        stemData = StemData()
        data = stemData.data

    tend0 = time.perf_counter_ns()
    print(f'Elapsed time reading stem data: {(tend0-tbegin)/1e9:.4f} seconds.')

    if readModelData:
        # Create ModelData object
        modelData = ModelData()
        symbolNames = ['u', 'upr', 'vak', 'OnUGlobal', 'TimeResol', \
                    'Qf_L', 'QTf', 'PfNet', 'FuelQty', 'QfDemandActual_L', 'EVak_L', \
                        'FuelCost', 'TotalCostU', 'TotalTaxUpr', 'StatsU', 'StatsTax']
        for symbolName in symbolNames:
            # logger.info(f'Reading symbol {symbolName}.')
            dfQf_Lcumcum = modelData[symbolName]
            # print(df)

    tend1 = time.perf_counter_ns()
    print(f'Elapsed time reading model data: {(tend1-tend0)/1e9:.4f} seconds.')

    # for symbolName in symbolNames:
    #     logger.info(f'Retrieving symbol {symbolName}.')
    #     df = modelData[symbolName]

    tend2 = time.perf_counter_ns()
    print(f'Elapsed time in total: {(tend2-tbegin)/1e9:.4f} seconds.')

#endregion Reading data


In [ ]:

#region Extracting data to show

# What is the reference for timestamps in the model? 
# Is it the time of the first record in the model?
# Convention: Timestamp represents the start of the time interval hence the first record is at time 0.

if readModelData:
    # Pick available plants using the u symbol and the OnUGlobal symbol
    dfTimeResol = modelData['TimeResol']
    timeIncr = (dfTimeResol['level'] / 60).to_numpy()
    timeVec = np.cumsum(timeIncr) - timeIncr[0]
    
    dfU = modelData['u']
    dfOnUGlobal = modelData['OnUGlobal']
    uAvail = dfOnUGlobal['u'].to_list()
    dfUpr = modelData['upr']

    # Remove columns of dfUpr that are not available
    dfUpr = dfUpr[dfUpr['u'].isin(uAvail)]
    orderU = ['MaNVak', 'MaVak', 'HoNVak', 'StVak', 'MaCool', 'MaCool2', 'MaAff1', 'MaAff2', 'MaBio', 'MaEk', 'MaNbk', 'MaNbKV', 'MaNEk', 'MaNhpAir', 'MaNhpPtX', 
        'HoNEk', 'HoNFlis', 'HoNhpAir', 'HoNhpArla', 'HoNhpBirn', 'HoNhpSew', 'HoGk', 'HoOk', 
        'StEk', 'StNEk', 'StNFlis', 'StNhpAir', 'StGk', 'StOk']

    # Setup the display order of plants.
    orderU = [u for u in orderU if u in uAvail]
    plantGroups = {'Ma': 'BHP', 'Ho': 'Holstebro', 'St': 'Struer'}


    # Fetch the records of Qf_L and create a pivot table
    dfQf_LRecs = modelData['Qf_L']
    # print(dfQf_LRecs.head())

    #region Abandoned code working on records of Qf_L
    # df = dfQf_LRecs.copy(deep=True)
    # df['time'] = [timeVec[int(tt[1:]) - 1] for tt in df.tt]
    # # df = df.sort_values(by=['time'])
    # # print(df.head())
    # # print(timeVec)
    # # print(timeIncr)

    # # Drop any row of df where the value of the column 'u' is not in uAvail
    # df = df[df['u'].isin(uAvail)]

    # # Also, replace values of df that are equal to 1E-14 with zero. The value 1E-14 is assigned within the GAMS model to ensure filled-in records.  
    # df = df.replace(1E-14, 0.0)
    # # print(df.head(20))

    # orderU = ['MaNVak', 'MaVak', 'HoNVak', 'StVak', 'MaCool', 'MaCool2', 'MaAff1', 'MaAff2', 'MaBio', 'MaEk', 'MaNbk', 'MaNbKV', 'MaNEk', 'MaNhpAir', 'MaNhpPtX', 
    #           'HoNEk', 'HoNFlis', 'HoNhpAir', 'HoNhpArla', 'HoNhpBirn', 'HoNhpSew', 'HoGk', 'HoOk', 
    #           'StEk', 'StNEk', 'StNFlis', 'StNhpAir', 'StGk', 'StOk']
    # orderU = [u for u in orderU if u in uAvail]
    # # print(orderU)

    # plantGroups = {'Ma': 'BHP', 'Ho': 'Holstebro', 'St': 'Struer'}
    # df['plantGroup'] = [plantGroups[u[:2]] for u in df['u']]
    # # print(df.head(20))

    # # Drop any row of df where the value of the column 'u' contains Cool. Cooled heat is not delivered to the district heating system.
    # df = df[~df['u'].str.contains('Cool')]
    # print(df.head(20))  

    # # Extract unique values of the column 'u' and sort them according to the order in orderU.
    # uUnique = df['u'].unique()
    # uUnique = [u for u in orderU if u in uUnique]
    # # print(f'{uUnique=}')

    # # # Sort df according to the order in column time, next to the order of orderU.
    # # df = df.sort_values(by=['time', 'u'])
    #endregion Abandoned code working on records of Qf_L

    # print(timeVec)
    print(dfQf_LRecs.head(20))
    dfQf_Lx = createPivot(dfQf_LRecs, indexName='tt', columnNames=['u'], valueName='level', createTimeColumn=True, timeVector=timeVec)
    
    # dfQf_Lavail nov contains a column name 'time' and a column for each plant that is available. Dimension 'tt' is used as index.
    # Pick only values of available production plants. 
    # Also, replace values of dfQf_Lavail that are less than 1E-12 with zero. The value 1E-14 is assigned within the GAMS model to ensure filled-in records.
    dfQf_L = dfQf_Lx[['time'] + uAvail]   # Pick only columns of available plants and the time column.
    # dfQf_L = dfQf_L.mask(dfQf_L.loc[:,:] < 1E-12 ,0.0, inplace=False)
    dfQf_L = dfQf_L.replace(1E-14, 0.0)


    # # If any column of dfQf_Lavail ends with 'Cool', reverse the sign of the column values. Cooled heat is not delivered to the district heating system.
    for col in dfQf_L.columns:
        if 'Cool' in col:
            dfQf_L.loc[:,col] = -dfQf_L.loc[:,col] 

    # Create a column in dfQf_L that holds the timestamp composed of the now plus the time column.
    timeOffset = datetime.fromisoformat('2024-01-08 00:00:00')
    dfQf_L['timestamp'] = [timeOffset + timedelta(hours=t) for t in dfQf_L['time']]

    # Sort columns of dfQf_L according to orderU and add the time column at the end.
    dfQf_L = dfQf_L[['time', 'timestamp'] + orderU]

    print(dfQf_L.head(20))

#endregion Extracting data to show

In [ ]:

# Plotly Express line (and scatter) plots have no option for stacking series. Therefore, the dataframe must be transformed.
# Create a dataframe from dfQf_L where each column is the sum of itself and the previous column.
# The first column is not changed and columns tt and time are not included in the summation.
# The result is a dataframe with the same dimensions as dfQf_L, but where each column is the sum of itself and all previous columns of dfQf_L.

def createStackedDf(df: pd.DataFrame, columns: list[str]) -> pd.DataFrame:
    """
    Creates a dataframe from df where each column contained in columns is the sum of itself and the previous column.
    Columns not in colums are not included in the summation.
    The result is a dataframe with the same dimensions as df, but where each column is the sum of itself and all previous columns of df.
    df: dataframe holding the data to be transformed. This dataframe is not changed.
    columns: list of column names to be included in the summation in the order they should appear in the result.

    """
    dfCum = df.copy()
    for i in range(1, len(columns)):
        dfCum[columns[i]] = dfCum[columns[i]] + dfCum[columns[i-1]]

    return dfCum


dfQf_Lcum = createStackedDf(dfQf_L, orderU)
print(f'df.head:\n{dfQf_Lcum.head(2)}')



In [ ]:
#region Setting up user interface

# App layout

# https://plotly.com/python-api-reference/

useStacked = False

df = dfQf_Lcum if useStacked else dfQf_L
titlePrefix = 'Akkum. varmeleverancer' if useStacked else 'Varmeproduktion'
# Drop columns containing Vak and Cool
df = df.drop(columns=[col for col in df.columns if 'Cool' in col])
# df = df.drop(columns=[col for col in df.columns if 'Vak' in col])
orderUlocal = [u for u in orderU if u in df.columns]
print(f'{orderUlocal=}')

# Plotly templates: https://plotly.com/python/templates/
# Plotly hovering: https://plotly.com/python/hover-text-and-formatting/

fig = px.line(df, 
            x="timestamp", 
            y=orderUlocal, 
            line_shape='hv',
            title=f'<b>{titlePrefix} fra {timeOffset:%Y-%m-%d}</b>',
            template='ggplot2',
            width=1200,
            height=15*len(df)
            )  

fig.layout.xaxis.title = 'Tid'
fig.layout.yaxis.title = f'Varmeproduktion [MWh/h]'
fig.layout.hovermode = 'closest'
fig.layout.legend.title = '<b>Anlæg</b>'
fig.show()


# https://plotly.com/python-api-reference/generated/plotly.graph_objects.Figure.html

pass


In [ ]:
app = Dash(__name__)

df = px.data.gapminder()
print(f'df.head: \n{df.head(10)}')

range_slider = dcc.RangeSlider(
    value=[1987, 2007],
    step=5,
    marks={i: str(i) for i in range(1952, 2012, 5)},
)

dtable = dash_table.DataTable(
    columns=[{"name": i, "id": i} for i in sorted(df.columns)],
    sort_action="native",
    page_size=10,
    style_table={"overflowX": "auto"},
)

download_button = html.Button("Download Filtered CSV", style={"marginTop": 20})
download_component = dcc.Download()

app.layout = html.Div(
    [
        html.H2("Gapminder data filtered download", style={"marginBottom": 20}),
        download_component,
        range_slider,
        download_button,
        dtable,
    ]
)


@app.callback(
    Output(dtable, "data"),
    Input(range_slider, "value"),
)
def update_table(slider_value):
    if not slider_value:
        return dash.no_update
    dff = df[df.year.between(slider_value[0], slider_value[1])]
    return dff.to_dict("records")


@app.callback(
    Output(download_component, "data"),
    Input(download_button, "n_clicks"),
    State(dtable, "derived_virtual_data"),
    prevent_initial_call=True,
)
def download_data(n_clicks, data):
    dff = pd.DataFrame(data)
    return dcc.send_data_frame(dff.to_csv, "filtered_csv.csv")

app.run(mode='inline', port=8056, debug=True)


In [ ]:
# Initialize the app
df = dfQf_L
cols = [c for c in df.columns if c in uAvail]
dfShort = df.copy(deep=True)
for c in cols:
    dfShort[c] = df[c].apply(lambda x: round(x, 2))

print(dfShort)

In [ ]:

def roundTable(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
    dfShort = df.copy(deep=True)
    for c in cols:
        dfShort[c] = df[c].apply(lambda x: round(x, 2))

    return dfShort

plantGroups = {'Ma': 'BHP', 'Ho': 'Holstebro', 'St': 'Struer'}
orderUlocal = [u for u in orderU if u in df.columns]
df = dfQf_Lcum if useStacked else dfQf_L
df = df[['timestamp'] + orderUlocal]
dfRound = roundTable(df, cols=[c for c in df.columns if c in uAvail])
dfRound['timestamp'] = dfRound['timestamp'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M'))
print(dfRound.head(20))

In [168]:
app = Dash(__name__, use_pages=False, external_stylesheets=[dbc.themes.MORPH])

# App layout
#region App layout timeseries
# app.layout = html.Div(
#     [
#         html.H4("Forsyningsselskabets varmeproduktion"),
#         # html.P("Anlæg: "),
#         # dcc.Checklist(
#         #     id="plants",
#         #     options=orderU,
#         #     value=orderU,
#         #     inline=True,
#         # ),
#         html.P("Gruppering: "),
#         dcc.RadioItems(
#             id="grouping",
#             options=["Grundlast", "SR", "Ingen"],
#             value="Ingen",
#             inline=True,
#         ),
#         dbc.Label('Submit'), html.Br(),
#         dbc.Button("Submit", id='buttonRun'),
#         dbc.RadioItems(options=[{'label': 'Unstacked', 'value': 1}, {'label': 'Stacked', 'value': 2}], value=1, id='radio'),
#         dcc.Graph(id="graph"),
#     ]
# )
#endregion App layout timeseries

checklist = dbc.Checklist(options=plantGroups, value=plantGroups.keys(), inline=True, id='selPlantGroups')

dtable = dash_table.DataTable(
    columns=[{"name": i, "id": i} for i in sorted(df.columns)],
    data = dfRound.to_dict("records"),
    page_size=10,
    style_table={"fontSize": 12, "overflowX": "auto"},
)

download_button = dbc.Button("Download Filtered Table as CSV", style={"marginTop": 20})
download_component = dcc.Download()

def download_data(n_clicks, data):
    dff = pd.DataFrame(data)
    return dcc.send_data_frame(dff.to_csv, "filtered_csv.csv")


#region App layout 
app.layout = html.Div(
    [
        dbc.Row([
        html.P(html.H1("App Header", style={'fontSize': '18', 'textAlign': 'center'})),
        html.Hr(style={'border': '2px solid black'}),
        ]),
        dbc.Row([
            dbc.Col(id='nav_bar', lg=2, children=['Nav bar']),
            dbc.Col([
                dbc.Tabs([
                    dbc.Tab([
                        html.Br(),
                        html.H4("Forsyningsselskabets varmeproduktion"),
                        # dbc.Label('Submit'), html.Br(),
                        dbc.Button("Submit", id='buttonRun'),
                        dbc.RadioItems(options=[{'label': 'Unstacked', 'value': 0}, {'label': 'Stacked', 'value': 1}], value=0, id='radio'),
                        dcc.Graph(id="graph"),
                        ], label='Timeseries'),

                    dbc.Tab([
                        html.Ul([
                            html.Br(),
                            html.Li('Number of Economies: 170'),
                            html.Li('Temporal Coverage: 1974 - 2019'),
                            html.Li('Update Frequency: Quarterly'),
                            html.Li('Last Updated: March 18, 2020'),
                            html.Li([
                                'Source: ',
                                html.A('https://datacatalog.worldbank.org/dataset/poverty-and-equity-database',
                                        href='https://datacatalog.worldbank.org/dataset/poverty-and-equity-database')
                            ])
                        ])
                    ], label='Intro'),
                    
                    dbc.Tab([
                        html.Br(),
                        html.P('Her specificeres lastplanlægningen', style={'textAlign': 'left'})
                    ], label='JobSpec'),

                    dbc.Tab([
                        html.Br(),
                        html.P('Her vises resultater for lastplanen', style={'fontSize': '22', 'textAlign': 'left'}), 
                        html.Div([
                            download_component,
                            # checklist,
                            download_button,
                            dtable,
                        ])                        
                    ], label='ResultsTable'),
                ])
            ], lg=10)
        ]),
        dbc.Row([
            html.Hr(style={'border': '2px solid black'}),
            html.P(html.H1("App Footer", style={'fontSize': '18', 'textAlign': 'center'})),
        ]),
    ], 
title='MockUI')

#endregion App layout

#region Callbacks

@app.callback(
    Output("graph", "figure"),
    Input("radio", "value")
)
def generate_chart(stacked):
    df = dfQf_L.copy(deep=True)
    # print(f'{plants=}')
    # print(f'{grouping=}')
    # uSelected = [u for u in orderU if u in plants]    # Sort according to predefined order.
    # df = dfQf_L[['time'] + uSelected]
    # Grouping ignored for now.
    if stacked:
        df = dfQf_Lcum
    else: 
        df = dfQf_L

    fig = px.line(df, 
            x="timestamp", 
            y=orderUlocal, 
            line_shape='hv',
            title=f'<b>{titlePrefix} fra {timeOffset:%Y-%m-%d}</b>',
            template='ggplot2',
            # width=1200,
            # height=15*len(df)
            )  

    fig.layout.xaxis.title = 'Tid'
    fig.layout.yaxis.title = f'Varmeproduktion [MWh/h]'
    fig.layout.hovermode = 'closest'
    fig.layout.legend.title = '<b>Anlæg</b>'
    # fig.show()

    return fig

@app.callback(
    Output(dtable, "data"),
    Input(checklist, "value"),
)
def update_table(groups):
    if not groups:
        return dash.no_update
    
    dff = df[df['plantGroup'].isin(groups)]
    cols = [c for c in dff.columns if c in uAvail]
    for c in cols:
        dff[c] = dff[c].apply(lambda x: round(x, 2))

    # Replace timestamp with a string representation.
    dff['timestamp'] = dff['timestamp'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M'))
    dff = dff[['timestamp'] + cols]

    return dff.to_dict("records")


@app.callback(
    Output(download_component, "data"),
    Input(download_button, "n_clicks"),
    State(dtable, "derived_virtual_data"),
    prevent_initial_call=True,
)
def download_data(n_clicks, data):
    dff = pd.DataFrame(data)
    return dcc.send_data_frame(dff.to_csv, "filtered_csv.csv")

#endregion Callbacks

# Run the app
app.run(mode='inline', port=8556)

#endregion Setting up user interface
pass